In [1]:
from sentence_transformers import CrossEncoder
import logging
import os
import pandas as pd
import torch, gc
from importlib import reload

import financerag.tasks.Base_Task; reload(financerag.tasks.Base_Task)
import financerag.tasks.ConvFinQATask; reload(financerag.tasks.ConvFinQATask)
import financerag.tasks.FinanceBenchTask; reload(financerag.tasks.FinanceBenchTask)
import financerag.tasks.FinDERTask; reload(financerag.tasks.FinDERTask)
import financerag.tasks.FinQATask; reload(financerag.tasks.FinQATask)
import financerag.tasks.FinQABenchTask; reload(financerag.tasks.FinQABenchTask)
import financerag.tasks.MultiHierttTask; reload(financerag.tasks.MultiHierttTask)
import financerag.tasks.TATQATask; reload(financerag.tasks.TATQATask)
reload(financerag.tasks)

from financerag.rerank import CrossEncoderReranker
from financerag.retrieval import DenseRetrieval, SentenceTransformerEncoder, BM25Retriever, HybridRetriever
from financerag.tasks import ConvFinQA, FinanceBench, FinDER, FinQA, FinQABench, MultiHiertt, TATQA

# Setup basic logging configuration to show info level messages.
logging.basicConfig(level=logging.INFO)

/opt/conda/envs/DSC/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
import os
import pandas as pd
results_dir = './financerag/results/'
time_obj = datetime.now()
subfolder = "submission_" + time_obj.strftime('%m%d%H%M')
output_dir = results_dir + subfolder
os.makedirs(output_dir, exist_ok=True)


combined_df = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': [25, 30, 35],
    'City': ['New York', 'Los Angeles', 'Chicago']
})
combined_df.to_csv(output_dir + '/' + subfolder + '.csv', index=False)

In [4]:
import torch
torch.cuda.set_device(1)
print("Current GPU:", torch.cuda.current_device())

Current GPU: 1


# Task

In [2]:
# Step 2: Initialize FinDER Task

convfinqa_task = ConvFinQA()
finbench_task = FinanceBench()
finder_task = FinDER()
finqa_task = FinQA()
finqabench_task = FinQABench()
multih_task = MultiHiertt()
tatqa_task = TATQA()

changed


In [ ]:
len(convfinqa_task.queries)

In [ ]:
421+

In [14]:
len(convfinqa_task.queries)

421

In [29]:
qu = pd.read_json("data/finqabench/queries.jsonl", lines=True)
qu['text'][0]

'What is the redemption price for the 0.875% 2025 Notes and the 1.375% 2029 Notes if they are redeemed prior to the applicable Par Call Date?\n\nredemption price, 0.875% Notes, 1.375% Notes, Par Call Date'

In [30]:
qu = pd.read_json("data/finbench/queries.jsonl", lines=True)
qu['text'][0]

'What is the FY2019 - FY2020 total revenue growth rate for Block (formerly known as Square)? Answer in units of percents and round to one decimal place. Approach the question asked by assuming the standpoint of an investment banking analyst who only has access to the statement of income.\n\ntotal revenue growth rate, Block, Square'

In [27]:
qu = pd.read_json("data/finqabench/queries_origin.jsonl", lines=True)
qu

,_id,title,text
0,q4aa0b116,,What is the redemption price for the 0.875% 20...
1,q4aa0a48c,,What is the maturity date of the 0.875% 2025 N...
2,q4aa0aeb4,,What are the conditions under which the Compan...
3,q4aa05fa4,,What is the Company's investment policy and st...
4,q4aa092f8,,What is the purpose of the Company's internal ...
...,...,...,...
95,q4aa01fda,,What risks are associated with the Company's i...
96,q4aa0a1d0,,What percentage of outstanding shares must vot...
97,q4aa0a34c,,What is the minimum percentage of shares requi...
98,q4aa05284,,What was the approximate dollar value of share...


# Initialize Model

In [10]:
# 줄임말 확장
# 테이블을 instruction: 테이블 컴포넌트를 뽑아달라고

In [3]:
# Step 3: Initialize DenseRetriever model

# You can replace 'intfloat/e5-large-v2' with any other model supported by SentenceTransformers.
# For example: 'BAAI/bge-large-en-v1.5', 'Linq-AI-Research/Linq-Embed-Mistral', etc.
base_encoder = "BAAI/bge-base-en-v1.5" #"BAAI/bge-m3" # "intfloat/multilingual-e5-large-instruct"  #"BAAI/bge-large-en-v1.5" #"nvidia/NV-Embed-v2"(20GB) "intfloat/e5-mistral-7b-instruct"(9GB) 
                                            #"dunzhang/stella_en_1.5B_v5" (6GB)  "jinaai/jina-embeddings-v3"(1.1GB) "jinaai/jina-embeddings-v2-base-code"(320MB)
encoder_model = SentenceTransformerEncoder(
    model_name_or_path=base_encoder,
    query_prompt='query: ',
    doc_prompt='passage: '
)

retrieval_model = DenseRetrieval(
     model=encoder_model
 )

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5


batch size: 8


In [8]:
print("Working on FinBench Task")
finqabench_results = finqabench_task.retrieve(
    retriever=retrieval_model,
    top_k = 10)



INFO:financerag.retrieval.dense:Encoding queries...


Working on FinBench Task


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

INFO:financerag.retrieval.dense:Sorting corpus by document length...
INFO:financerag.retrieval.dense:Encoding corpus in batches... This may take a while.
INFO:financerag.retrieval.dense:Encoding batch 1/1...


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

In [10]:
for doc_id, score in finqabench_task.retrieve_results.items():
    print(doc_id, score)

q4aa0b116 {'d4aa05d24': 0.6745860576629639, 'd4aa07b56': 0.6752417087554932, 'd4aa0a9e6': 0.6891423463821411, 'd4aa0a7d4': 0.6909683346748352, 'd4aa075a2': 0.6869709491729736, 'd4aa0b1f2': 0.801344633102417, 'd4aa0a52c': 0.7031479477882385, 'd4aa0a66c': 0.6956996321678162, 'd4aa1afee': 0.6925300359725952, 'd4aa0839e': 0.693058967590332}
q4aa0a48c {'d4aa06274': 0.7180978655815125, 'd4aa05798': 0.7207592129707336, 'd4aa0a66c': 0.7360135912895203, 'd4aa0660c': 0.7266372442245483, 'd4aa0a7d4': 0.7240824699401855, 'd4aa0985c': 0.7964701652526855, 'd4aa06a44': 0.7470573782920837, 'd4aa068fa': 0.7427541017532349, 'd4aa067ba': 0.7316091060638428, 'd4aa0a52c': 0.7638562917709351}
q4aa0aeb4 {'d4aa080e2': 0.6598387360572815, 'd4aa04474': 0.6610119342803955, 'd4aa04834': 0.6757684350013733, 'd4aa07db8': 0.669884204864502, 'd4aa19d7e': 0.6639583110809326, 'd4aa05ebe': 0.6948834657669067, 'd4aa04e60': 0.6814190149307251, 'd4aa0abd0': 0.6975031495094299, 'd4aa08ad8': 0.6828032732009888, 'd4aa0af90': 

In [21]:
print(len(finder_task.retrieve_results['q00001']))

80


In [14]:
# 결과를 DataFrame으로 불러오기
results_df = [
    convfinqa_task.load_results(),
    finbench_task.load_results(),
]

# DataFrame을 하나로 합치기
combined_df = pd.concat(results_df, ignore_index=True)

# 결과 확인
print(combined_df)


       query_id  corpus_id     score
0     qd4982518  dd4c4c208  0.761090
1     qd4982518  dd4ba3af4  0.753088
2     qd4982518  dd4b89cbc  0.745192
3     qd4982518  dd4c4fb38  0.743647
4     qd4982518  dd4bf6f9c  0.743375
...         ...        ...       ...
5205  q4aa0b3fa  dd2b07a56  0.727264
5206  q4aa0b3fa  dd2b1453a  0.727212
5207  q4aa0b3fa  dd2ad1a32  0.726640
5208  q4aa0b3fa  dd2ac1c4a  0.726012
5209  q4aa0b3fa  dd2ad662c  0.725589

[5210 rows x 3 columns]


In [50]:
len(convfinqa_task.corpus), len(convfinqa_task.retrieve_results)

(2066, 421)

In [20]:
list(convfinqa_task.queries.keys())[0]

'qd4982518'

In [36]:
conf_results_ex = convfinqa_task.retrieve_results['qd4982518']
conf_results_ex1 = {k: v for k, v in sorted(conf_results_ex.items(), key=lambda item: item[1], reverse=False)}

In [ ]:
len(convfinqa_task.corpus), len(convfinqa_task.retrieve_results)

In [4]:
# bm25_retriever = BM25Retriever()
# dense_retriever = DenseRetrieval(model=encoder_model)

# # Initialize the hybrid retriever
# hybrid_retriever = HybridRetriever(
#     lexical_retriever=bm25_retriever,
#     dense_retriever=dense_retriever,
#     lexical_weight=0.4,
#     dense_weight=0.6
# )

batch size: 16


In [4]:
torch.cuda.empty_cache()

In [8]:
# Step 4: Perform retrieval

print("Working on ConvfinQA Task")
convfinqa_result = convfinqa_task.retrieve(
    retriever=retrieval_model)
torch.cuda.empty_cache()
print("Working on FinBench Task")
finbench_result = finbench_task.retrieve(
    retriever=retrieval_model)
torch.cuda.empty_cache()
print("Working on FinDER Task")
finder_result = finder_task.retrieve(
    retriever=retrieval_model)
torch.cuda.empty_cache()
print("Working on FinQA Task")
finqa_result = finqa_task.retrieve(
    retriever=retrieval_model)
torch.cuda.empty_cache()
print("Working on FinQABench Task")
finqabench_result = finqabench_task.retrieve(
    retriever=retrieval_model)
torch.cuda.empty_cache()
print("Working on MultiHiertt Task")
multih_result = multih_task.retrieve(
    retriever=retrieval_model)
torch.cuda.empty_cache()
print("Working on TATQA Task")
tatqa_result = tatqa_task.retrieve(
    retriever=retrieval_model)
torch.cuda.empty_cache()

results = [
    convfinqa_result,
    finbench_result,
    finder_result,
    finqa_result,
    finqabench_result,
    multih_result,
    tatqa_result
]

INFO:financerag.retrieval.dense:Encoding queries...


Working on ConvfinQA Task


Batches:   0%|          | 0/53 [00:00<?, ?it/s]

INFO:financerag.retrieval.dense:Sorting corpus by document length...
INFO:financerag.retrieval.dense:Encoding corpus in batches... This may take a while.
INFO:financerag.retrieval.dense:Encoding batch 1/1...


Batches:   0%|          | 0/259 [00:00<?, ?it/s]

...

Working on FinDER Task


Batches:   0%|          | 0/27 [00:00<?, ?it/s]

INFO:financerag.retrieval.dense:Sorting corpus by document length...
INFO:financerag.retrieval.dense:Encoding corpus in batches... This may take a while.
INFO:financerag.retrieval.dense:Encoding batch 1/1...


Batches:   0%|          | 0/1733 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# # Print a portion of the retrieval results to verify the output.
# for result in results:
#     print(f"Retrieved results for {len(result)} queries. Here's an example of the top 5 documents for the first query:")

# for result in results:
#     for q_id, res in result.items():
#         print(f"\nQuery ID: {q_id}")
#         # Sort the result to print the top 5 document ID and its score
#         sorted_results = sorted(res.items(), key=lambda x: x[1], reverse=True)

#         for i, (doc_id, score) in enumerate(sorted_results[:5]):
#             print(f"  Document {i + 1}: Document ID = {doc_id}, Score = {score}")

#         break  # Only show the first query

## Evaluation

In [40]:
def get_evalset(dataset_name):
    qrels = {}
    df_qrels = pd.read_csv(f"./data/test/{dataset_name}_qrels.tsv", sep='\t')
    for _, row in df_qrels.iterrows():
        if row['query_id'] not in qrels:
            qrels[row['query_id']] = {}
        qrels[row['query_id']][row['corpus_id']] = row['score']
    return qrels

In [41]:
tasks = [
    convfinqa_task,
    finbench_task,
    finder_task,
    finqa_task,
    finqabench_task,
    multih_task,
    tatqa_task
]

qrels = [
    get_evalset('ConvFinQA'),
    get_evalset('FinanceBench'),
    get_evalset('FinDER'),
    get_evalset('FinQA'),
    get_evalset('FinQABench'),
    get_evalset('MultiHeirtt'),
    get_evalset('TATQA')
]

# for qrel, task in zip(qrels, tasks):
#     metrics = task.evaluate(qrels=qrel, results=task.retrieve_results, k_values=[10])
#     retrieve_ndcgs.append()

In [81]:
import matplotlib.pyplot as plt

ndcg_values = []
map_values = []
recall_values = []
precision_values = []

tasks = [
    convfinqa_task,
    finbench_task

]

qrels = [
    get_evalset('ConvFinQA'),
    get_evalset('FinanceBench')
]

dataset_names = [
    'ConvFinQA',
    'FinanceBench'
]

for qrel, task in zip(qrels, tasks):

    metrics = task.evaluate(qrels=qrel, results=task.retrieve_results, k_values=[10])

    ndcg_values.append(metrics[0]['NDCG@10'])  # NDCG@10
    map_values.append(metrics[1]['MAP@10'])    # MAP@10
    recall_values.append(metrics[2]['Recall@10'])  # Recall@10
    precision_values.append(metrics[3]['P@10'])  # P@10

plt.figure(figsize=(12, 8))
plt.plot(ndcg_values, marker='o', label='NDCG@10', color='b')
plt.title('NDCG@10')
plt.xlabel('Dataset')
plt.ylabel('Score')
plt.xticks(range(len(dataset_names)), dataset_names, rotation=45)  # X축에 데이터셋 이름을 설정
plt.grid(True)
plt.tight_layout()
plt.show()

INFO:financerag.tasks.Base_Task:NDCG@10: 0.1238


126
0


ZeroDivisionError: float division by zero

In [8]:
import matplotlib.pyplot as plt

ndcg_values = []
map_values = []
recall_values = []
precision_values = []

dataset_names = [
    'ConvFinQA',
    'FinanceBench',
    'FinDER',
    'FinQA',
    'FinQABench',
    'MultiHeirtt',
    'TATQA'
]

for qrel, task in zip(qrels, tasks):

    metrics = task.evaluate(qrels=qrel, results=task.retrieve_results, k_values=[10])

    ndcg_values.append(metrics[0]['NDCG@10'])  # NDCG@10
    map_values.append(metrics[1]['MAP@10'])    # MAP@10
    recall_values.append(metrics[2]['Recall@10'])  # Recall@10
    precision_values.append(metrics[3]['P@10'])  # P@10

plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(ndcg_values, marker='o', label='NDCG@10', color='b')
plt.title('NDCG@10')
plt.xlabel('Dataset')
plt.ylabel('Score')
plt.xticks(range(len(dataset_names)), dataset_names, rotation=45)  # X축에 데이터셋 이름을 설정
plt.grid(True)

plt.subplot(2, 2, 2)
plt.plot(map_values, marker='o', label='MAP@10', color='g')
plt.title('MAP@10')
plt.xlabel('Dataset')
plt.ylabel('Score')
plt.xticks(range(len(dataset_names)), dataset_names, rotation=45)  # X축에 데이터셋 이름을 설정
plt.grid(True)

plt.subplot(2, 2, 3)
plt.plot(recall_values, marker='o', label='Recall@10', color='r')
plt.title('Recall@10')
plt.xlabel('Dataset')
plt.ylabel('Score')
plt.xticks(range(len(dataset_names)), dataset_names, rotation=45)  # X축에 데이터셋 이름을 설정
plt.grid(True)

plt.subplot(2, 2, 4)
plt.plot(precision_values, marker='o', label='P@10', color='c')
plt.title('Precision@10')
plt.xlabel('Dataset')
plt.ylabel('Score')
plt.xticks(range(len(dataset_names)), dataset_names, rotation=45)  # X축에 데이터셋 이름을 설정
plt.grid(True)

plt.tight_layout()
plt.show()

INFO:financerag.tasks.Base_Task:NDCG@10: 0.3429


ZeroDivisionError: float division by zero

# Initialize Reranker

"jinaai/jina-reranker-v2-base-multilingual",
            "Alibaba-NLP/gte-multilingual-reranker-base",
        "BAAI/bge-reranker-v2-m3"

In [5]:
# Step 5: Initialize CrossEncoder Reranker
# base_reranker1 = "jinaai/jina-reranker-v2-base-multilingual"  # Model name
# reranker1 = CrossEncoderReranker(
#     model=CrossEncoder(base_reranker1, trust_remote_code=True)  # Ensure trust_remote_code is enabled
# )

base_reranker = "BAAI/bge-reranker-v2-m3" #"BAAI/bge-reranker-base" #m2 

reranker = CrossEncoderReranker(
    model=CrossEncoder(base_reranker)
)


INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cuda


In [11]:
torch.cuda.empty_cache()

# Perform Reranking (1)

In [6]:
# Step 6: Perform rerankin

top_k= 100 # Number of Reranking results
batch_size = 8 # 32

print("\nWorking on ConvFinQA Reranking")
convfinqa_rerank = convfinqa_task.rerank(
    reranker=reranker,
    results=convfinqa_result,
    top_k=top_k,  # Rerank the top 100 documents
    batch_size=batch_size
)
torch.cuda.empty_cache()
print("\nWorking on FinBench Reranking")
finbench_rerank = finbench_task.rerank(
    reranker=reranker,
    results=finbench_result,
    top_k=top_k,  # Rerank the top 100 documents
    batch_size=batch_size
)
torch.cuda.empty_cache()
print("\nWorking on FinDER Reranking")
finder_rerank = finder_task.rerank(
    reranker=reranker,
    results=finder_result,
    top_k=top_k,  # Rerank the top 100 documents
    batch_size=batch_size
)
torch.cuda.empty_cache()
print("\nWorking on FinQA Reranking")
finqa_rerank = finqa_task.rerank(
    reranker=reranker,
    results=finqa_result,
    top_k=top_k,  # Rerank the top 100 documents
    batch_size=batch_size
)
torch.cuda.empty_cache()
print("\nWorking on FinQABench Reranking")
finqabench_rerank = finqabench_task.rerank(
    reranker=reranker,
    results=finqabench_result,
    top_k=top_k,  # Rerank the top 100 documents
    batch_size=batch_size
)
torch.cuda.empty_cache()
print("\nWorking on MultiHiertt Reranking")
multih_rerank = multih_task.rerank(
    reranker=reranker,
    results=multih_result,
    top_k=top_k,  # Rerank the top 100 documents
    batch_size=batch_size
)
torch.cuda.empty_cache()
print("\nWorking on TATQA Reranking")
tatqa_rerank = tatqa_task.rerank(
    reranker=reranker,
    results=tatqa_result,
    top_k=top_k,  # Rerank the top 100 documents
    batch_size=batch_size
)
torch.cuda.empty_cache()
reranking_results = [
    convfinqa_rerank,
    finbench_rerank,
    finder_rerank,
    finqa_rerank,
    finqabench_rerank,
    multih_rerank,
    tatqa_rerank
]



Working on ConvFinQA Reranking


INFO:financerag.rerank.cross_encoder:Starting To Rerank Top-100....


Batches:   0%|          | 0/5263 [00:00<?, ?it/s]

INFO:financerag.rerank.cross_encoder:Starting To Rerank Top-100....



Working on FinBench Reranking


...

INFO:financerag.rerank.cross_encoder:Starting To Rerank Top-100....


Batches:   0%|          | 0/12175 [00:00<?, ?it/s]


Working on TATQA Reranking


INFO:financerag.rerank.cross_encoder:Starting To Rerank Top-100....


Batches:   0%|          | 0/20788 [00:00<?, ?it/s]

In [11]:
# for i, rerank_result in enumerate(reranking_results):
#     print(f"\nReranking results for Task {i + 1} ({len(rerank_result)} queries). Here's an example of the top 5 documents for the first query:")

#     for q_id, result in rerank_result.items():
#         print(f"\nQuery ID: {q_id}")
        
#         sorted_results = sorted(result.items(), key=lambda x: x[1], reverse=True)

#         for i, (doc_id, score) in enumerate(sorted_results[:5]):
#             print(f"  Document {i + 1}: Document ID = {doc_id}, Score = {score}")

#         break  # Only show the first query for each task

## Evaluation

In [13]:
import matplotlib.pyplot as plt

# 각 평가지표를 저장할 리스트
retrieval_ndcg_values = []
rerank_ndcg_values = []

retrieval_map_values = []
rerank_map_values = []

retrieval_recall_values = []
rerank_recall_values = []

retrieval_precision_values = []
rerank_precision_values = []

# 데이터셋 이름 리스트
dataset_names = [
    'ConvFinQA',
    'FinanceBench',
    'FinDER',
    'FinQA',
    'FinQABench',
    'MultiHeirtt',
    'TATQA'
]

for qrel, task in zip(qrels, tasks):
    # 평가지표 평가 (retrieval 결과)
    #retrieval_metrics = task.evaluate(qrels=qrel, results=task.retrieve_results, k_values=[10])
    
    # 평가지표 평가 (rerank 결과)
    rerank_metrics = task.evaluate(qrels=qrel, results=task.rerank_results, k_values=[10])

    # retrieval_ndcg_values.append(retrieval_metrics[0]['NDCG@10'])
    # retrieval_map_values.append(retrieval_metrics[1]['MAP@10'])
    # retrieval_recall_values.append(retrieval_metrics[2]['Recall@10'])
    # retrieval_precision_values.append(retrieval_metrics[3]['P@10'])
    
    rerank_ndcg_values.append(rerank_metrics[0]['NDCG@10'])
    rerank_map_values.append(rerank_metrics[1]['MAP@10'])
    rerank_recall_values.append(rerank_metrics[2]['Recall@10'])
    rerank_precision_values.append(rerank_metrics[3]['P@10'])

# 그래프 생성
plt.figure(figsize=(14, 10))

# NDCG@10 시각화
#plt.subplot(2, 2, 1)
#plt.plot(retrieval_ndcg_values, marker='o', label='Retrieval NDCG@10', color='b')
plt.plot(rerank_ndcg_values, marker='x', label='Rerank NDCG@10', color='b', linestyle='--')
plt.title('NDCG@10 Comparison')
plt.xlabel('Dataset')
plt.ylabel('Score')
plt.xticks(range(len(dataset_names)), dataset_names, rotation=45)  # X축에 데이터셋 이름을 설정
plt.legend()
plt.grid(True)

# # MAP@10 시각화
# plt.subplot(2, 2, 2)
# plt.plot(retrieval_map_values, marker='o', label='Retrieval MAP@10', color='g')
# plt.plot(rerank_map_values, marker='x', label='Rerank MAP@10', color='g', linestyle='--')
# plt.title('MAP@10 Comparison')
# plt.xlabel('Dataset')
# plt.ylabel('Score')
# plt.xticks(range(len(dataset_names)), dataset_names, rotation=45)  # X축에 데이터셋 이름을 설정
# plt.legend()
# plt.grid(True)

# # Recall@10 시각화
# plt.subplot(2, 2, 3)
# plt.plot(retrieval_recall_values, marker='o', label='Retrieval Recall@10', color='r')
# plt.plot(rerank_recall_values, marker='x', label='Rerank Recall@10', color='r', linestyle='--')
# plt.title('Recall@10 Comparison')
# plt.xlabel('Dataset')
# plt.ylabel('Score')
# plt.xticks(range(len(dataset_names)), dataset_names, rotation=45)  # X축에 데이터셋 이름을 설정
# plt.legend()
# plt.grid(True)

# # Precision@10 시각화
# plt.subplot(2, 2, 4)
# plt.plot(retrieval_precision_values, marker='o', label='Retrieval P@10', color='c')
# plt.plot(rerank_precision_values, marker='x', label='Rerank P@10', color='c', linestyle='--')
# plt.title('Precision@10 Comparison')
# plt.xlabel('Dataset')
# plt.ylabel('Score')
# plt.xticks(range(len(dataset_names)), dataset_names, rotation=45)  # X축에 데이터셋 이름을 설정
# plt.legend()
# plt.grid(True)

plt.tight_layout()
plt.show()

INFO:financerag.tasks.Base_Task:NDCG@10: 0.4805


ZeroDivisionError: float division by zero

# Perform Reranking (2)

In [14]:
# Step 5: Initialize CrossEncoder Reranker
base_reranker2 = "BAAI/bge-reranker-v2-m3" #"BAAI/bge-reranker-v2-m3" #"jinaai/jina-reranker-v2-base-multilingual" #'cross-encoder/ms-marco-MiniLM-L-12-v2' #"BAAI/bge-reranker-base"

reranker2 = CrossEncoderReranker(
    model=CrossEncoder(base_reranker2)
)

INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cuda


In [15]:
# Step 6: Perform reranking

top_k= 20 # Number of Reranking results
batch_size = 8 # 32

print("\nWorking on ConvFinQA Reranking")
convfinqa_rerank_second = convfinqa_task.rerank(
    reranker=reranker2,
    results=convfinqa_rerank,
    top_k=top_k,  # Rerank the top 100 documents
    batch_size=batch_size
)
torch.cuda.empty_cache()
print("\nWorking on FinBench Reranking")
finbench_rerank_second = finbench_task.rerank(
    reranker=reranker2,
    results=finbench_rerank,
    top_k=top_k,  # Rerank the top 100 documents
    batch_size=batch_size
)
torch.cuda.empty_cache()
print("\nWorking on FinDER Reranking")
finder_rerank_second = finder_task.rerank(
    reranker=reranker2,
    results=finder_rerank,
    top_k=top_k,  # Rerank the top 100 documents
    batch_size=batch_size
)
torch.cuda.empty_cache()
print("\nWorking on FinQA Reranking")
finqa_rerank_second = finqa_task.rerank(
    reranker=reranker2,
    results=finqa_rerank,
    top_k=top_k,  # Rerank the top 100 documents
    batch_size=batch_size
)
torch.cuda.empty_cache()
print("\nWorking on FinQABench Reranking")
finqabench_rerank_second = finqabench_task.rerank(
    reranker=reranker2,
    results=finqabench_rerank,
    top_k=top_k,  # Rerank the top 100 documents
    batch_size=batch_size
)
torch.cuda.empty_cache()
print("\nWorking on MultiHiertt Reranking")
multih_rerank_second = multih_task.rerank(
    reranker=reranker2,
    results=multih_rerank,
    top_k=top_k,  # Rerank the top 100 documents
    batch_size=batch_size
)
torch.cuda.empty_cache()
print("\nWorking on TATQA Reranking")
tatqa_rerank_second = tatqa_task.rerank(
    reranker=reranker2,
    results=tatqa_rerank,
    top_k=top_k,  # Rerank the top 100 documents
    batch_size=batch_size
)
torch.cuda.empty_cache()
reranking_results_second = [
    convfinqa_rerank_second,
    finbench_rerank_second,
    finder_rerank_second,
    finqa_rerank_second,
    finqabench_rerank_second,
    multih_rerank_second,
    tatqa_rerank_second
]


INFO:financerag.rerank.cross_encoder:Starting To Rerank Top-20....



Working on ConvFinQA Reranking


Batches:   0%|          | 0/1053 [00:00<?, ?it/s]

INFO:financerag.rerank.cross_encoder:Starting To Rerank Top-20....



Working on FinBench Reranking


...

INFO:financerag.rerank.cross_encoder:Starting To Rerank Top-20....


Batches:   0%|          | 0/2435 [00:00<?, ?it/s]

INFO:financerag.rerank.cross_encoder:Starting To Rerank Top-20....



Working on TATQA Reranking


Batches:   0%|          | 0/4158 [00:00<?, ?it/s]

## Evaluation

In [22]:
import matplotlib.pyplot as plt

rerank_second_ndcg_values = []
rerank_second_map_values = []
rerank_second_recall_values = []
rerank_second_precision_values = []

dataset_names = [
    'ConvFinQA',
    'FinanceBench',
    'FinDER',
    'FinQA',
    'FinQABench',
    'MultiHeirtt',
    'TATQA'
]

for qrel, task in zip(qrels, tasks):
    rerank_second_metrics = task.evaluate(qrels=qrel, results=task.rerank_results, k_values=[10])
    rerank_second_ndcg_values.append(rerank_second_metrics[0]['NDCG@10'])
    rerank_second_map_values.append(rerank_second_metrics[1]['MAP@10'])
    rerank_second_recall_values.append(rerank_second_metrics[2]['Recall@10'])
    rerank_second_precision_values.append(rerank_second_metrics[3]['P@10'])

plt.figure(figsize=(14, 10))

plt.subplot(2, 2, 1)
plt.plot(retrieval_ndcg_values, marker='o', label='Retrieval NDCG@10', color='b')
plt.plot(rerank_ndcg_values, marker='x', label='Rerank NDCG@10', color='b', linestyle='--')
plt.plot(rerank_second_ndcg_values, marker='s', label='Rerank Second NDCG@10', color='b', linestyle='-.')
plt.title('NDCG@10 Comparison')
plt.xlabel('Dataset')
plt.ylabel('Score')
plt.xticks(range(len(dataset_names)), dataset_names, rotation=45)
plt.legend()
plt.grid(True)

plt.subplot(2, 2, 2)
plt.plot(retrieval_map_values, marker='o', label='Retrieval MAP@10', color='g')
plt.plot(rerank_map_values, marker='x', label='Rerank MAP@10', color='g', linestyle='--')
plt.plot(rerank_second_map_values, marker='s', label='Rerank Second MAP@10', color='g', linestyle='-.')
plt.title('MAP@10 Comparison')
plt.xlabel('Dataset')
plt.ylabel('Score')
plt.xticks(range(len(dataset_names)), dataset_names, rotation=45)
plt.legend()
plt.grid(True)

plt.subplot(2, 2, 3)
plt.plot(retrieval_recall_values, marker='o', label='Retrieval Recall@10', color='r')
plt.plot(rerank_recall_values, marker='x', label='Rerank Recall@10', color='r', linestyle='--')
plt.plot(rerank_second_recall_values, marker='s', label='Rerank Second Recall@10', color='r', linestyle='-.')
plt.title('Recall@10 Comparison')
plt.xlabel('Dataset')
plt.ylabel('Score')
plt.xticks(range(len(dataset_names)), dataset_names, rotation=45)
plt.legend()
plt.grid(True)

plt.subplot(2, 2, 4)
plt.plot(retrieval_precision_values, marker='o', label='Retrieval P@10', color='c')
plt.plot(rerank_precision_values, marker='x', label='Rerank P@10', color='c', linestyle='--')
plt.plot(rerank_second_precision_values, marker='s', label='Rerank Second P@10', color='c', linestyle='-.')
plt.title('Precision@10 Comparison')
plt.xlabel('Dataset')
plt.ylabel('Score')
plt.xticks(range(len(dataset_names)), dataset_names, rotation=45)
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

INFO:financerag.tasks.Base_Task:NDCG@10: 0.5377


ZeroDivisionError: float division by zero

In [ ]:
## reranker 3

In [19]:
# Step 5: Initialize CrossEncoder Reranker
base_reranker3 = "BAAI/bge-reranker-v2-m3" #"BAAI/bge-reranker-v2-m3" #"jinaai/jina-reranker-v2-base-multilingual" #'cross-encoder/ms-marco-MiniLM-L-12-v2' #"BAAI/bge-reranker-base"

reranker3 = CrossEncoderReranker(
    model=CrossEncoder(base_reranker2)
)

INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cuda


# Save Results

In [16]:
os.getcwd()

'/data/DSC/Financerag'

In [7]:
# Step 7: Save results
results_dir = './financerag/results/'
subfolder = 'submission_12020100'

output_dir = results_dir+subfolder
convfinqa_task.save_results(output_dir=output_dir)
finbench_task.save_results(output_dir=output_dir)
finder_task.save_results(output_dir=output_dir)
finqa_task.save_results(output_dir=output_dir)
finqabench_task.save_results(output_dir=output_dir)
multih_task.save_results(output_dir=output_dir)
tatqa_task.save_results(output_dir=output_dir)

csv_files = [
    output_dir + '/ConvFinQA/results.csv',
    output_dir + '/FinanceBench/results.csv',
    output_dir + '/FinDER/results.csv',
    output_dir + '/FinQA/results.csv',
    output_dir + '/FinQABench/results.csv',
    output_dir + '/MultiHiertt/results.csv',
    output_dir + '/TAT-QA/results.csv'
]

results_df = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(results_df, ignore_index=False)
combined_df.to_csv(output_dir+'/'+subfolder+'.csv', index=False)

INFO:financerag.tasks.Base_Task:Output directory set to: ./financerag/results/submission_12020100/ConvFinQA
INFO:financerag.tasks.Base_Task:Output directory set to: ./financerag/results/submission_12020100/FinanceBench
INFO:financerag.tasks.Base_Task:Output directory set to: ./financerag/results/submission_12020100/FinDER
INFO:financerag.tasks.Base_Task:Output directory set to: ./financerag/results/submission_12020100/FinQA
INFO:financerag.tasks.Base_Task:Output directory set to: ./financerag/results/submission_12020100/FinQABench
INFO:financerag.tasks.Base_Task:Output directory set to: ./financerag/results/submission_12020100/MultiHiertt
INFO:financerag.tasks.Base_Task:Output directory set to: ./financerag/results/submission_12020100/TAT-QA


In [44]:
os.getcwd()

'c:\\Users\\God_YJ\\interns\\DS_COMP\\FinanceRAG-main'

In [18]:
sample = pd.read_csv('../data/sample_submission.csv')
sample.head()

,query_id,corpus_id
0,qd496c6a0,dd4b92b32
1,qd496c6a0,dd4ba2a5a
2,qd496c6a0,dd4be1f98
3,qd496c6a0,dd4ba07d2
4,qd496c6a0,dd4ba02f0


In [21]:
# sample과 combined_df에서 고유한 query_id 값 얻기
sample_query_ids = sample['query_id'].unique()
combined_query_ids = combined_df['query_id'].unique()

# sample과 combined_df에만 존재하는 query_id 값 구하기
sample_only_query_ids = list(set(sample_query_ids) - set(combined_query_ids))
combined_only_query_ids = list(set(combined_query_ids) - set(sample_query_ids))

# 결과 출력
print("query_id present only in sample:")
print(sample_only_query_ids)

print("query_id present only in combined_df:")
print(combined_only_query_ids)


query_id present only in sample:
['query_id']
query_id present only in combined_df:
['q00097', 'qd2abb228', 'q00029']
